## 6.4. 多输入多输出通道
### 6.4.1 多输入通道
对于多通道数如$c_i$的输入，卷积和也要相应的$c_i$维度，每个维度上的二维张量做卷积，然后$c_i$对应结果维度相加即可得到最后的结果
![Snipaste_2023-08-11_17-30-37](https://cdn.staticaly.com/gh/SisyphusTang/Picture-bed@master/20230811/Snipaste_2023-08-11_17-30-37.3a8f2jnn6sg0.png)


### 6.4.2 多输出通道
假设$c_i$是输入通道，$c_0$是输出通道，卷积核的形状应该为 $c_0 × c_i × k_h × k_w$


### 6.4.3 1×1卷积层
把不同通道的特征进行一个加权和 得到输出
![Snipaste_2023-08-11_22-10-03](https://cdn.staticaly.com/gh/SisyphusTang/Picture-bed@master/20230811/Snipaste_2023-08-11_22-10-03.1qdoo0rj4hsw.webp)


### 二维卷积层总结
![多通道复杂度](https://cdn.staticaly.com/gh/SisyphusTang/Picture-bed@master/20230811/多通道复杂度.2zpfi5rnqm00.webp)

In [3]:
import torch
from d2l import torch as d2l


def corr2d(X,K):
    h,w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1),(X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w] * K).sum()
    return Y


def corr2d_multi_in(X,K):
     # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(corr2d(x,k) for x,k in zip(X,K))







In [20]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [13]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [21]:
# 沿着维度0堆叠K K+1 K+2
#print(K.shape)  # 2 2 2

K = torch.stack((K , K + 1, K + 2), 0) # 新加的一维为

K.shape  # 3 2 2 2

torch.Size([3, 2, 2, 2])

In [7]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [23]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))


X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6